## Weekly Data Wrangling - Organic Social

In [63]:
import pandas as pd
import numpy as np
import datetime as dt
from time import strptime

### Twitter

Read in in platform data export and wrangle.

In [64]:
# week of:
twitter = pd.read_csv('twitter_weekly_pull.csv')
print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 40 columns):
Tweet id                        116 non-null float64
Tweet permalink                 116 non-null object
Tweet text                      116 non-null object
time                            116 non-null object
impressions                     116 non-null int64
engagements                     116 non-null int64
engagement rate                 116 non-null float64
retweets                        116 non-null int64
replies                         116 non-null int64
likes                           116 non-null int64
user profile clicks             116 non-null int64
url clicks                      116 non-null int64
hashtag clicks                  116 non-null int64
detail expands                  116 non-null int64
permalink clicks                116 non-null int64
app opens                       116 non-null int64
app installs                    116 non-null int64
follows              

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1.072540e+18,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,2018-12-11 17:17 +0000,15831,1115,0.070431,27,2,255,...,-,-,-,-,-,-,-,-,-,-
1,1.072170e+18,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,2018-12-10 17:00 +0000,126587,12874,0.101701,129,25,1110,...,-,-,-,-,-,-,-,-,-,-
2,1.072010e+18,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",2018-12-10 06:00 +0000,14432,309,0.021411,12,17,168,...,-,-,-,-,-,-,-,-,-,-
3,1.072010e+18,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,2018-12-10 05:58 +0000,12383,499,0.040297,27,5,114,...,-,-,-,-,-,-,-,-,-,-
4,1.072010e+18,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,2018-12-10 05:50 +0000,14097,754,0.053487,22,9,116,...,-,-,-,-,-,-,-,-,-,-


In [65]:
# filter out unneeded columns
twitter = twitter.filter(['Tweet permalink','Tweet text','time','impressions','engagements','engagement rate','retweets','replies','likes','url clicks','media views','media engagements','promoted impressions','promoted media views','promoted media engagements'])
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 15 columns):
Tweet permalink               116 non-null object
Tweet text                    116 non-null object
time                          116 non-null object
impressions                   116 non-null int64
engagements                   116 non-null int64
engagement rate               116 non-null float64
retweets                      116 non-null int64
replies                       116 non-null int64
likes                         116 non-null int64
url clicks                    116 non-null int64
media views                   116 non-null int64
media engagements             116 non-null int64
promoted impressions          116 non-null object
promoted media views          116 non-null object
promoted media engagements    116 non-null object
dtypes: float64(1), int64(8), object(6)
memory usage: 13.7+ KB


In [66]:
# eliminate reply tweets
twitter = twitter[~twitter['Tweet text'].str.startswith('@')]
twitter['Tweet text'].unique()

array(['.@KrisJenner called in the big guns to cheer up Koko’s first night away from True. Catch up on #KUWTK On Demand: https://t.co/nEKGYUwSTG https://t.co/yk94mt2kOz',
       'The fam has been #blessed this past year 🙏 #KUWTK https://t.co/D5YgDJ4EU6',
       "It's been real, West Coast ✌️ Try not to miss us too much 😘",
       'RT if this season of #KUWTK has been absolutely magical ✨ https://t.co/mX0xPDFygN',
       'The moment Alice first met @KimKardashian 🤗❤️ #KUWTK https://t.co/fJxeB32Lto',
       "Actual footage of Alice Johnson finding out she's 👏 a 👏 free 👏 woman 👏 @KimKardashian #KUWTK https://t.co/Os1bl6iZ18",
       "There was only one way to get Alice Johnson free and @KimKardashian didn't hesitate to do it 💪 #KUWTK https://t.co/UUgMjMx3fE",
       "Me: Don't take photos of me\n*Someone starts recording a story*\nMe: \n\n#KUWTK https://t.co/6x2LidNGaj",
       'RT if your ❤️ beats for @KhloeKardashian #KUWTK https://t.co/nFcm7iCfok',
       'Have you heard about the #KUW

In [67]:
# rename columns
twitter = twitter.rename(columns = {'Tweet permalink':'link','Tweet text':'message','time':'date','impressions':'org imp','engagements':'engagement','engagement rate':'ER','retweets':'retweets/shares','replies':'replies/comments','url clicks':'link clicks','media views':'org video views','media engagements':'org video completes','promoted impressions':'paid imp','promoted media views':'paid video views','promoted media engagements':'paid video completes'})
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 115
Data columns (total 15 columns):
link                    48 non-null object
message                 48 non-null object
date                    48 non-null object
org imp                 48 non-null int64
engagement              48 non-null int64
ER                      48 non-null float64
retweets/shares         48 non-null int64
replies/comments        48 non-null int64
likes                   48 non-null int64
link clicks             48 non-null int64
org video views         48 non-null int64
org video completes     48 non-null int64
paid imp                48 non-null object
paid video views        48 non-null object
paid video completes    48 non-null object
dtypes: float64(1), int64(8), object(6)
memory usage: 6.0+ KB


In [68]:
# change data types
twitter['date'] = pd.to_datetime(twitter['date'])
twitter['date'] = twitter['date'].dt.normalize()
twitter['paid imp'] = twitter['paid imp'].map(lambda x: "0" if x == '-' else None)
twitter['paid video views'] = twitter['paid video views'].map(lambda x: "0" if x == '-' else None)
twitter['paid video completes'] = twitter['paid video completes'].map(lambda x: "0" if x == '-' else None)
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 115
Data columns (total 15 columns):
link                    48 non-null object
message                 48 non-null object
date                    48 non-null datetime64[ns]
org imp                 48 non-null int64
engagement              48 non-null int64
ER                      48 non-null float64
retweets/shares         48 non-null int64
replies/comments        48 non-null int64
likes                   48 non-null int64
link clicks             48 non-null int64
org video views         48 non-null int64
org video completes     48 non-null int64
paid imp                48 non-null object
paid video views        48 non-null object
paid video completes    48 non-null object
dtypes: datetime64[ns](1), float64(1), int64(8), object(5)
memory usage: 6.0+ KB


In [69]:
print(twitter['paid imp'].unique())
twitter.head()

['0']


,link,message,date,org imp,engagement,ER,retweets/shares,replies/comments,likes,link clicks,org video views,org video completes,paid imp,paid video views,paid video completes
0,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,2018-12-11,15831,1115,0.070431,27,2,255,7,3317,767,0,0,0
1,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,2018-12-10,126587,12874,0.101701,129,25,1110,20,30321,11024,0,0,0
2,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",2018-12-10,14432,309,0.021411,12,17,168,0,0,0,0,0,0
3,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,2018-12-10,12383,499,0.040297,27,5,114,1,2960,293,0,0,0
4,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,2018-12-10,14097,754,0.053487,22,9,116,1,2543,528,0,0,0


In [70]:
#convert paid columns to integer
twitter['paid imp'] = twitter['paid imp'].astype(int)
twitter['paid video views'] = twitter['paid video views'].astype(int)
twitter['paid video completes'] = twitter['paid video completes'].astype(int)
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 115
Data columns (total 15 columns):
link                    48 non-null object
message                 48 non-null object
date                    48 non-null datetime64[ns]
org imp                 48 non-null int64
engagement              48 non-null int64
ER                      48 non-null float64
retweets/shares         48 non-null int64
replies/comments        48 non-null int64
likes                   48 non-null int64
link clicks             48 non-null int64
org video views         48 non-null int64
org video completes     48 non-null int64
paid imp                48 non-null int64
paid video views        48 non-null int64
paid video completes    48 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(11), object(2)
memory usage: 6.0+ KB


In [71]:
# create new columns
twitter['platform'] = 'Twitter'
twitter['media type'] = np.nan
twitter['content bucket'] = np.nan
twitter['theme'] = np.nan
twitter['subtheme'] = np.nan
twitter['subtheme2'] = np.nan
twitter['objective'] = np.nan
twitter['targ aud'] = np.nan
twitter['CTA'] = np.nan
twitter['destination'] = np.nan
twitter['paid'] = twitter['paid imp'].map(lambda x: "paid" if x > 0 else "org")
twitter['reach'] = np.nan
twitter['vis eng'] = twitter['retweets/shares'] + twitter['replies/comments'] + twitter['likes']
twitter['vis eng rate'] = twitter['vis eng']/(twitter['org imp'] + twitter['paid imp'])
twitter['VR'] = (twitter['org video views'] + twitter['paid video views'])/(twitter['org imp'] + twitter['paid imp'])
twitter['CR'] = (twitter['org video completes'] + twitter['paid video completes'])/(twitter['org imp'] + twitter['paid imp'])
print(twitter.info())
twitter.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 115
Data columns (total 31 columns):
link                    48 non-null object
message                 48 non-null object
date                    48 non-null datetime64[ns]
org imp                 48 non-null int64
engagement              48 non-null int64
ER                      48 non-null float64
retweets/shares         48 non-null int64
replies/comments        48 non-null int64
likes                   48 non-null int64
link clicks             48 non-null int64
org video views         48 non-null int64
org video completes     48 non-null int64
paid imp                48 non-null int64
paid video views        48 non-null int64
paid video completes    48 non-null int64
platform                48 non-null object
media type              0 non-null float64
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null flo

,link,message,date,org imp,engagement,ER,retweets/shares,replies/comments,likes,link clicks,...,objective,targ aud,CTA,destination,paid,reach,vis eng,vis eng rate,VR,CR
0,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,2018-12-11,15831,1115,0.070431,27,2,255,7,...,NaN,NaN,NaN,NaN,org,NaN,284,0.017939,0.209526,0.048449
1,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,2018-12-10,126587,12874,0.101701,129,25,1110,20,...,NaN,NaN,NaN,NaN,org,NaN,1264,0.009985,0.239527,0.087086
2,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",2018-12-10,14432,309,0.021411,12,17,168,0,...,NaN,NaN,NaN,NaN,org,NaN,197,0.013650,0.000000,0.000000
3,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,2018-12-10,12383,499,0.040297,27,5,114,1,...,NaN,NaN,NaN,NaN,org,NaN,146,0.011790,0.239037,0.023661
4,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,2018-12-10,14097,754,0.053487,22,9,116,1,...,NaN,NaN,NaN,NaN,org,NaN,147,0.010428,0.180393,0.037455


In [72]:
# reorder columns
twitter = twitter[['platform','date','link','message','media type','content bucket','theme','subtheme','subtheme2','objective','targ aud','CTA','destination','paid','reach','org imp','paid imp','engagement','ER','link clicks','retweets/shares','likes','replies/comments','vis eng','vis eng rate','org video views','paid video views','org video completes','paid video completes','VR','CR']]
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 115
Data columns (total 31 columns):
platform                48 non-null object
date                    48 non-null datetime64[ns]
link                    48 non-null object
message                 48 non-null object
media type              0 non-null float64
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float64
objective               0 non-null float64
targ aud                0 non-null float64
CTA                     0 non-null float64
destination             0 non-null float64
paid                    48 non-null object
reach                   0 non-null float64
org imp                 48 non-null int64
paid imp                48 non-null int64
engagement              48 non-null int64
ER                      48 non-null float64
link clicks             48 non-null int64
retweets/shares         48 non-

### Facebook 

Read in in platform data export and wrangle.

In [73]:
fb = pd.read_csv('fb_weekly_pull.csv')
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 54 columns):
Post ID                                                                        11 non-null object
Permalink                                                                      11 non-null object
Post Message                                                                   11 non-null object
Type                                                                           11 non-null object
Countries                                                                      0 non-null float64
Languages                                                                      0 non-null float64
Posted                                                                         11 non-null object
Audience Targeting                                                             11 non-null object
Lifetime Post Total Reach                                                      12 non-null object
Lifetime Post o

,Post ID,Permalink,Post Message,Type,Countries,Languages,Posted,Audience Targeting,Lifetime Post Total Reach,Lifetime Post organic reach,...,Lifetime Matched Audience Targeting Consumptions by Type - video play,Lifetime Matched Audience Targeting Consumptions by Type - other clicks,Lifetime Matched Audience Targeting Consumptions by Type - link clicks,Lifetime Matched Audience Targeting Consumptions by Type - photo view,Lifetime Negative Feedback from Users by Type - hide_all_clicks,Lifetime Negative Feedback from Users by Type - hide_clicks,Lifetime Negative Feedback from Users by Type - report_spam_clicks,Lifetime Negative Feedback by Type - hide_all_clicks,Lifetime Negative Feedback by Type - hide_clicks,Lifetime Negative Feedback by Type - report_spam_clicks
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lifetime: The number of people who had your Pa...,Lifetime: The number of people who had your Pa...,...,Lifetime: The number of clicks anywhere in the...,NaN,NaN,NaN,Lifetime: The number of times people have give...,NaN,NaN,Lifetime: The number of people who have given ...,NaN,NaN
1,166720031891_10155869324316892,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,Video,NaN,NaN,12/11/18 9:17,,326548,326548,...,7487,25213.0,111.0,NaN,13,18.0,NaN,13,18.0,NaN
2,166720031891_10155861355461892,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,Video,NaN,NaN,12/10/18 9:00,,2291676,358586,...,49692,32064.0,NaN,NaN,15,28.0,1.0,15,28.0,1.0
3,166720031891_10155861301676892,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",Video,NaN,NaN,12/9/18 17:00,,256686,256686,...,5307,14081.0,NaN,NaN,19,21.0,NaN,19,21.0,NaN
4,166720031891_10155861335196892,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",Photo,NaN,NaN,12/9/18 10:30,,381003,381003,...,NaN,2842.0,NaN,10035.0,29,51.0,NaN,28,34.0,NaN


In [74]:
# drop unwanted rows and columns
fb.drop(fb.index[0], inplace=True)
fb = fb.filter(['Permalink','Post Message','Type','Posted','Lifetime Post organic reach','Lifetime Post Organic Impressions','Lifetime Post Paid Impressions','Lifetime Engaged Users','Lifetime Organic views to 95%.1','Lifetime Paid views to 95%.1','Lifetime Organic Video Views.1','Lifetime Paid Video Views.1','Lifetime Talking About This (Post) by action type - like','Lifetime Talking About This (Post) by action type - comment','Lifetime Talking About This (Post) by action type - share','Lifetime Matched Audience Targeting Consumptions by Type - link clicks'])
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 11
Data columns (total 16 columns):
Permalink                                                                 11 non-null object
Post Message                                                              11 non-null object
Type                                                                      11 non-null object
Posted                                                                    11 non-null object
Lifetime Post organic reach                                               11 non-null object
Lifetime Post Organic Impressions                                         11 non-null object
Lifetime Post Paid Impressions                                            11 non-null object
Lifetime Engaged Users                                                    11 non-null object
Lifetime Organic views to 95%.1                                           11 non-null object
Lifetime Paid views to 95%.1                                

,Permalink,Post Message,Type,Posted,Lifetime Post organic reach,Lifetime Post Organic Impressions,Lifetime Post Paid Impressions,Lifetime Engaged Users,Lifetime Organic views to 95%.1,Lifetime Paid views to 95%.1,Lifetime Organic Video Views.1,Lifetime Paid Video Views.1,Lifetime Talking About This (Post) by action type - like,Lifetime Talking About This (Post) by action type - comment,Lifetime Talking About This (Post) by action type - share,Lifetime Matched Audience Targeting Consumptions by Type - link clicks
1,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,Video,12/11/18 9:17,326548,417931,0,24177,12891,0,118720,0,903.0,19.0,18,111.0
2,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,Video,12/10/18 9:00,358586,459958,2378097,67721,11914,39965,133013,1077570,2580.0,128.0,103,NaN
3,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",Video,12/9/18 17:00,256686,332642,0,15054,39460,0,94689,0,838.0,226.0,48,NaN
4,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",Photo,12/9/18 10:30,381003,641661,0,10446,0,0,0,0,615.0,73.0,38,NaN
5,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,Video,12/9/18 9:00,475160,602424,2662769,141440,11735,76025,148224,1178813,3954.0,330.0,280,8.0


In [75]:
# rename columns
fb = fb.rename(columns = {'Permalink':'link','Post Message':'message','Type':'media type','Posted':'date','Lifetime Post organic reach':'reach','Lifetime Post Organic Impressions':'org imp','Lifetime Post Paid Impressions':'paid imp','Lifetime Engaged Users':'engagement','Lifetime Talking About This (Post) by action type - share':'retweets/shares','Lifetime Talking About This (Post) by action type - comment':'replies/comments','Lifetime Talking About This (Post) by action type - like':'likes','Lifetime Matched Audience Targeting Consumptions by Type - link clicks':'link clicks','Lifetime Organic Video Views.1':'org video views','Lifetime Organic views to 95%.1':'org video completes','Lifetime Paid Video Views.1':'paid video views','Lifetime Paid views to 95%.1':'paid video completes'})
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 11
Data columns (total 16 columns):
link                    11 non-null object
message                 11 non-null object
media type              11 non-null object
date                    11 non-null object
reach                   11 non-null object
org imp                 11 non-null object
paid imp                11 non-null object
engagement              11 non-null object
org video completes     11 non-null object
paid video completes    11 non-null object
org video views         11 non-null object
paid video views        11 non-null object
likes                   11 non-null float64
replies/comments        11 non-null float64
retweets/shares         11 non-null object
link clicks             8 non-null float64
dtypes: float64(3), object(13)
memory usage: 1.5+ KB


In [76]:
# change data types
fb['date'] = pd.to_datetime(fb['date'])
fb['date'] = fb['date'].dt.normalize()

fb['link clicks'] = fb['link clicks'].fillna(0)

cols = ['reach','org imp','paid imp','engagement','org video completes','paid video completes','org video views','paid video views','likes','replies/comments','retweets/shares','link clicks']

for col in cols:
    fb[col] = fb[col].astype(int)

print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 11
Data columns (total 16 columns):
link                    11 non-null object
message                 11 non-null object
media type              11 non-null object
date                    11 non-null datetime64[ns]
reach                   11 non-null int64
org imp                 11 non-null int64
paid imp                11 non-null int64
engagement              11 non-null int64
org video completes     11 non-null int64
paid video completes    11 non-null int64
org video views         11 non-null int64
paid video views        11 non-null int64
likes                   11 non-null int64
replies/comments        11 non-null int64
retweets/shares         11 non-null int64
link clicks             11 non-null int64
dtypes: datetime64[ns](1), int64(12), object(3)
memory usage: 1.5+ KB
None


,link,message,media type,date,reach,org imp,paid imp,engagement,org video completes,paid video completes,org video views,paid video views,likes,replies/comments,retweets/shares,link clicks
1,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,Video,2018-12-11,326548,417931,0,24177,12891,0,118720,0,903,19,18,111
2,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,Video,2018-12-10,358586,459958,2378097,67721,11914,39965,133013,1077570,2580,128,103,0
3,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",Video,2018-12-09,256686,332642,0,15054,39460,0,94689,0,838,226,48,0
4,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",Photo,2018-12-09,381003,641661,0,10446,0,0,0,0,615,73,38,0
5,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,Video,2018-12-09,475160,602424,2662769,141440,11735,76025,148224,1178813,3954,330,280,8


In [77]:
# create new columns
fb['platform'] = 'Facebook'
fb['content bucket'] = np.nan
fb['theme'] = np.nan
fb['subtheme'] = np.nan
fb['subtheme2'] = np.nan
fb['objective'] = np.nan
fb['targ aud'] = np.nan
fb['CTA'] = np.nan
fb['destination'] = np.nan
fb['paid'] = fb['paid imp'].map(lambda x: "paid" if x > 0 else "org")
fb['ER'] = fb['engagement']/(fb['org imp'] + fb['paid imp'])
fb['vis eng'] = fb['retweets/shares'] + fb['replies/comments'] + fb['likes']
fb['vis eng rate'] = fb['vis eng']/(fb['org imp'] + fb['paid imp'])
fb['VR'] = (fb['org video views'] + fb['paid video views'])/(fb['org imp'] + fb['paid imp'])
fb['CR'] = (fb['org video completes'] + fb['paid video completes'])/(fb['org imp'] + fb['paid imp'])
print(fb.info())
fb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 11
Data columns (total 31 columns):
link                    11 non-null object
message                 11 non-null object
media type              11 non-null object
date                    11 non-null datetime64[ns]
reach                   11 non-null int64
org imp                 11 non-null int64
paid imp                11 non-null int64
engagement              11 non-null int64
org video completes     11 non-null int64
paid video completes    11 non-null int64
org video views         11 non-null int64
paid video views        11 non-null int64
likes                   11 non-null int64
replies/comments        11 non-null int64
retweets/shares         11 non-null int64
link clicks             11 non-null int64
platform                11 non-null object
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float6

,link,message,media type,date,reach,org imp,paid imp,engagement,org video completes,paid video completes,...,objective,targ aud,CTA,destination,paid,ER,vis eng,vis eng rate,VR,CR
1,https://www.facebook.com/kuwtk/posts/101558693...,No one knows how to make Khloé 😂 quite like Th...,Video,2018-12-11,326548,417931,0,24177,12891,0,...,NaN,NaN,NaN,NaN,org,0.057849,940,0.002249,0.284066,0.030845
2,https://www.facebook.com/kuwtk/posts/101558613...,A bigger family means even bigger changes for ...,Video,2018-12-10,358586,459958,2378097,67721,11914,39965,...,NaN,NaN,NaN,NaN,paid,0.023862,2811,0.000990,0.426554,0.018280
3,https://www.facebook.com/kuwtk/posts/101558613...,"Dolls, who do you think was the MOST exciting ...",Video,2018-12-09,256686,332642,0,15054,39460,0,...,NaN,NaN,NaN,NaN,org,0.045256,1112,0.003343,0.284657,0.118626
4,https://www.facebook.com/kuwtk/posts/101558613...,"‘Tis the holiday season, so tag that bestie yo...",Photo,2018-12-09,381003,641661,0,10446,0,0,...,NaN,NaN,NaN,NaN,org,0.016280,726,0.001131,0.000000,0.000000
5,https://www.facebook.com/kuwtk/posts/101558590...,It’s not every day that you can change someone...,Video,2018-12-09,475160,602424,2662769,141440,11735,76025,...,NaN,NaN,NaN,NaN,paid,0.043318,4564,0.001398,0.406419,0.026877


In [78]:
# reorder columns
fb = fb[['platform','date','link','message','media type','content bucket','theme','subtheme','subtheme2','objective','targ aud','CTA','destination','paid','reach','org imp','paid imp','engagement','ER','link clicks','retweets/shares','likes','replies/comments','vis eng','vis eng rate','org video views','paid video views','org video completes','paid video completes','VR','CR']]
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 11
Data columns (total 31 columns):
platform                11 non-null object
date                    11 non-null datetime64[ns]
link                    11 non-null object
message                 11 non-null object
media type              11 non-null object
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float64
objective               0 non-null float64
targ aud                0 non-null float64
CTA                     0 non-null float64
destination             0 non-null float64
paid                    11 non-null object
reach                   11 non-null int64
org imp                 11 non-null int64
paid imp                11 non-null int64
engagement              11 non-null int64
ER                      11 non-null float64
link clicks             11 non-null int64
retweets/shares         11 non-nu

### Instagram

Read in Tracer data export and wrangle.

In [79]:
ig = pd.read_csv('ig_weekly_pull.csv')
print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
Page Name              11 non-null object
Post Created Date      11 non-null object
Post Permalink         10 non-null object
Post Media Type        10 non-null object
Post Caption           10 non-null object
Reach                  10 non-null float64
Impressions            10 non-null float64
Post Comments Count    10 non-null float64
Post Like Count        10 non-null float64
Engagement             10 non-null float64
Video Views            10 non-null float64
dtypes: float64(6), object(5)
memory usage: 1.1+ KB
None


,Page Name,Post Created Date,Post Permalink,Post Media Type,Post Caption,Reach,Impressions,Post Comments Count,Post Like Count,Engagement,Video Views
0,The Guest Book,12/16/18,https://www.instagram.com/p/BrdSI4NAJb8/,VIDEO,Here’s a few hints at what @mrmattwalsh & @BKB...,939.0,1110.0,2.0,59.0,62.0,397.0
1,The Guest Book,12/17/18,https://www.instagram.com/p/Brf5VHtH-Q8/,VIDEO,Your imaginary vacay is waiting. Screenshot ou...,692.0,831.0,0.0,40.0,42.0,205.0
2,The Guest Book,12/18/18,https://www.instagram.com/p/Brg0rRtAvup/,VIDEO,We interrupt your scroll to bring you backstag...,810.0,1001.0,1.0,53.0,56.0,320.0
3,The Guest Book,12/18/18,https://www.instagram.com/p/BrgoV_9gLty/,VIDEO,Dave’s love life is absolute trash. So he’s tu...,828.0,1020.0,1.0,45.0,48.0,411.0
4,The Guest Book,12/18/18,https://www.instagram.com/p/BriPTd5n0rF/,CAROUSEL_ALBUM,You wouldn’t believe how long it took to get e...,1127.0,1956.0,2.0,107.0,112.0,0.0


In [80]:
# drop unwanted rows and columns
ig.drop(columns = ['Page Name'], axis=1, inplace=True)
ig.drop(ig.index[-2:], inplace=True)
ig

,Post Created Date,Post Permalink,Post Media Type,Post Caption,Reach,Impressions,Post Comments Count,Post Like Count,Engagement,Video Views
0,12/16/18,https://www.instagram.com/p/BrdSI4NAJb8/,VIDEO,Here’s a few hints at what @mrmattwalsh & @BKB...,939.0,1110.0,2.0,59.0,62.0,397.0
1,12/17/18,https://www.instagram.com/p/Brf5VHtH-Q8/,VIDEO,Your imaginary vacay is waiting. Screenshot ou...,692.0,831.0,0.0,40.0,42.0,205.0
2,12/18/18,https://www.instagram.com/p/Brg0rRtAvup/,VIDEO,We interrupt your scroll to bring you backstag...,810.0,1001.0,1.0,53.0,56.0,320.0
3,12/18/18,https://www.instagram.com/p/BrgoV_9gLty/,VIDEO,Dave’s love life is absolute trash. So he’s tu...,828.0,1020.0,1.0,45.0,48.0,411.0
4,12/18/18,https://www.instagram.com/p/BriPTd5n0rF/,CAROUSEL_ALBUM,You wouldn’t believe how long it took to get e...,1127.0,1956.0,2.0,107.0,112.0,0.0
5,12/18/18,https://www.instagram.com/p/BripV5JnG8Z/,IMAGE,The Barefeet Retreat has seen a lot of action ...,817.0,966.0,7.0,43.0,50.0,0.0
6,12/19/18,https://www.instagram.com/p/BrjNR6HnqHo/,IMAGE,Everyone loves a good drama llama. #TheGuestBook,1001.0,1196.0,3.0,93.0,98.0,0.0
7,12/19/18,https://www.instagram.com/p/BrlJRVgAxbN/,VIDEO,We’ve got 3 reasons for you to catch up on #Th...,649.0,775.0,2.0,44.0,49.0,235.0
8,12/20/18,https://www.instagram.com/p/Brl1BMynLir/,VIDEO,Ugh reality 🙄 Relive Dave’s magical night when...,948.0,1146.0,5.0,69.0,77.0,466.0
9,12/20/18,https://www.instagram.com/p/Brnmzaoggsg/,VIDEO,"Careful, Gabe. Donny's got eyes & ears everywh...",525.0,630.0,0.0,37.0,37.0,198.0


In [81]:
#rename columns
ig.columns = ['date','link','media type','message','org imp','reach','engagement','likes','replies/comments','org video views']
ig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 10 columns):
date                10 non-null object
link                10 non-null object
media type          10 non-null object
message             10 non-null object
org imp             10 non-null float64
reach               10 non-null float64
engagement          10 non-null float64
likes               10 non-null float64
replies/comments    10 non-null float64
org video views     10 non-null float64
dtypes: float64(6), object(4)
memory usage: 880.0+ bytes


In [82]:
# change data types
ig['date'] = pd.to_datetime(ig['date'])
ig['date'] = ig['date'].dt.normalize()

cols = ['reach','org imp','engagement','org video views','likes','replies/comments']

for col in cols:
    ig[col] = ig[col].astype(int)

print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 10 columns):
date                10 non-null datetime64[ns]
link                10 non-null object
media type          10 non-null object
message             10 non-null object
org imp             10 non-null int64
reach               10 non-null int64
engagement          10 non-null int64
likes               10 non-null int64
replies/comments    10 non-null int64
org video views     10 non-null int64
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 880.0+ bytes
None


,date,link,media type,message,org imp,reach,engagement,likes,replies/comments,org video views
0,2018-12-16,https://www.instagram.com/p/BrdSI4NAJb8/,VIDEO,Here’s a few hints at what @mrmattwalsh & @BKB...,939,1110,2,59,62,397
1,2018-12-17,https://www.instagram.com/p/Brf5VHtH-Q8/,VIDEO,Your imaginary vacay is waiting. Screenshot ou...,692,831,0,40,42,205
2,2018-12-18,https://www.instagram.com/p/Brg0rRtAvup/,VIDEO,We interrupt your scroll to bring you backstag...,810,1001,1,53,56,320
3,2018-12-18,https://www.instagram.com/p/BrgoV_9gLty/,VIDEO,Dave’s love life is absolute trash. So he’s tu...,828,1020,1,45,48,411
4,2018-12-18,https://www.instagram.com/p/BriPTd5n0rF/,CAROUSEL_ALBUM,You wouldn’t believe how long it took to get e...,1127,1956,2,107,112,0


In [83]:
# create new columns
ig['platform'] = 'Instagram'
ig['content bucket'] = np.nan
ig['theme'] = np.nan
ig['subtheme'] = np.nan
ig['subtheme2'] = np.nan
ig['objective'] = np.nan
ig['targ aud'] = np.nan
ig['CTA'] = np.nan
ig['destination'] = np.nan
ig['paid'] = "org"
ig['paid imp'] = 0
ig['ER'] = ig['engagement']/(ig['org imp'] + ig['paid imp'])
ig['link clicks'] = 0
ig['retweets/shares'] = 0
ig['vis eng'] = ig['retweets/shares'] + ig['replies/comments'] + ig['likes']
ig['vis eng rate'] = ig['vis eng']/(ig['org imp'] + ig['paid imp'])
ig['paid video views'] = 0
ig['org video completes'] = 0
ig['paid video completes'] = 0
ig['VR'] = (ig['org video views'] + ig['paid video views'])/(ig['org imp'] + ig['paid imp'])
ig['CR'] = (ig['org video completes'] + ig['paid video completes'])/(ig['org imp'] + ig['paid imp'])
print(ig.info())
ig.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 31 columns):
date                    10 non-null datetime64[ns]
link                    10 non-null object
media type              10 non-null object
message                 10 non-null object
org imp                 10 non-null int64
reach                   10 non-null int64
engagement              10 non-null int64
likes                   10 non-null int64
replies/comments        10 non-null int64
org video views         10 non-null int64
platform                10 non-null object
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float64
objective               0 non-null float64
targ aud                0 non-null float64
CTA                     0 non-null float64
destination             0 non-null float64
paid                    10 non-null object
paid imp                10 non-null 

,date,link,media type,message,org imp,reach,engagement,likes,replies/comments,org video views,...,ER,link clicks,retweets/shares,vis eng,vis eng rate,paid video views,org video completes,paid video completes,VR,CR
0,2018-12-16,https://www.instagram.com/p/BrdSI4NAJb8/,VIDEO,Here’s a few hints at what @mrmattwalsh & @BKB...,939,1110,2,59,62,397,...,0.002130,0,0,121,0.128860,0,0,0,0.422790,0.0
1,2018-12-17,https://www.instagram.com/p/Brf5VHtH-Q8/,VIDEO,Your imaginary vacay is waiting. Screenshot ou...,692,831,0,40,42,205,...,0.000000,0,0,82,0.118497,0,0,0,0.296243,0.0
2,2018-12-18,https://www.instagram.com/p/Brg0rRtAvup/,VIDEO,We interrupt your scroll to bring you backstag...,810,1001,1,53,56,320,...,0.001235,0,0,109,0.134568,0,0,0,0.395062,0.0
3,2018-12-18,https://www.instagram.com/p/BrgoV_9gLty/,VIDEO,Dave’s love life is absolute trash. So he’s tu...,828,1020,1,45,48,411,...,0.001208,0,0,93,0.112319,0,0,0,0.496377,0.0
4,2018-12-18,https://www.instagram.com/p/BriPTd5n0rF/,CAROUSEL_ALBUM,You wouldn’t believe how long it took to get e...,1127,1956,2,107,112,0,...,0.001775,0,0,219,0.194321,0,0,0,0.000000,0.0


In [84]:
# reorder columns
ig = ig[['platform','date','link','message','media type','content bucket','theme','subtheme','subtheme2','objective','targ aud','CTA','destination','paid','reach','org imp','paid imp','engagement','ER','link clicks','retweets/shares','likes','replies/comments','vis eng','vis eng rate','org video views','paid video views','org video completes','paid video completes','VR','CR']]
ig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 31 columns):
platform                10 non-null object
date                    10 non-null datetime64[ns]
link                    10 non-null object
message                 10 non-null object
media type              10 non-null object
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float64
objective               0 non-null float64
targ aud                0 non-null float64
CTA                     0 non-null float64
destination             0 non-null float64
paid                    10 non-null object
reach                   10 non-null int64
org imp                 10 non-null int64
paid imp                10 non-null int64
engagement              10 non-null int64
ER                      10 non-null float64
link clicks             10 non-null int64
retweets/shares         10 non-nul

In [85]:
# concatenate all dataframes
all_data = pd.concat([twitter,fb,ig], ignore_index=True)
print(all_data.info())
all_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 31 columns):
platform                69 non-null object
date                    69 non-null datetime64[ns]
link                    69 non-null object
message                 69 non-null object
media type              21 non-null object
content bucket          0 non-null float64
theme                   0 non-null float64
subtheme                0 non-null float64
subtheme2               0 non-null float64
objective               0 non-null float64
targ aud                0 non-null float64
CTA                     0 non-null float64
destination             0 non-null float64
paid                    69 non-null object
reach                   21 non-null float64
org imp                 69 non-null int64
paid imp                69 non-null int64
engagement              69 non-null int64
ER                      69 non-null float64
link clicks             69 non-null int64
retweets/shares         69 non-

,platform,date,link,message,media type,content bucket,theme,subtheme,subtheme2,objective,...,likes,replies/comments,vis eng,vis eng rate,org video views,paid video views,org video completes,paid video completes,VR,CR
0,Twitter,2018-12-11,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,NaN,NaN,NaN,NaN,NaN,NaN,...,255,2,284,0.017939,3317,0,767,0,0.209526,0.048449
1,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,NaN,NaN,NaN,NaN,NaN,NaN,...,1110,25,1264,0.009985,30321,0,11024,0,0.239527,0.087086
2,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",NaN,NaN,NaN,NaN,NaN,NaN,...,168,17,197,0.013650,0,0,0,0,0.000000,0.000000
3,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,NaN,NaN,NaN,NaN,NaN,NaN,...,114,5,146,0.011790,2960,0,293,0,0.239037,0.023661
4,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,NaN,NaN,NaN,NaN,NaN,NaN,...,116,9,147,0.010428,2543,0,528,0,0.180393,0.037455


In [87]:
all_data['overall index'] = (all_data['vis eng rate']/(all_data['vis eng'].sum()/(all_data['org imp'].sum() + all_data['paid imp'].sum())))*100
all_data.head()

,platform,date,link,message,media type,content bucket,theme,subtheme,subtheme2,objective,...,replies/comments,vis eng,vis eng rate,org video views,paid video views,org video completes,paid video completes,VR,CR,overall index
0,Twitter,2018-12-11,https://twitter.com/KUWTK/status/1072540945810...,.@KrisJenner called in the big guns to cheer u...,NaN,NaN,NaN,NaN,NaN,NaN,...,2,284,0.017939,3317,0,767,0,0.209526,0.048449,652.930068
1,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072174102960...,The fam has been #blessed this past year 🙏 #KU...,NaN,NaN,NaN,NaN,NaN,NaN,...,25,1264,0.009985,30321,0,11024,0,0.239527,0.087086,363.424870
2,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072008006353...,"It's been real, West Coast ✌️ Try not to miss ...",NaN,NaN,NaN,NaN,NaN,NaN,...,17,197,0.013650,0,0,0,0,0.000000,0.000000,496.816926
3,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072007502898...,RT if this season of #KUWTK has been absolutel...,NaN,NaN,NaN,NaN,NaN,NaN,...,5,146,0.011790,2960,0,293,0,0.239037,0.023661,429.124846
4,Twitter,2018-12-10,https://twitter.com/KUWTK/status/1072005489897...,The moment Alice first met @KimKardashian 🤗❤️ ...,NaN,NaN,NaN,NaN,NaN,NaN,...,9,147,0.010428,2543,0,528,0,0.180393,0.037455,379.531051


In [92]:
platforms = all_data.groupby('platform')[['vis eng','org imp','paid imp']].sum().reset_index()
platforms

,platform,vis eng,org imp,paid imp
0,Facebook,19561,4866906,6857588
1,Instagram,1221,8336,0
2,Twitter,20293,3216931,0


In [97]:
def platform_index(all_data):
    if all_data['platform'] == 'Facebook':
        all_data['platform index'] = (all_data['vis eng rate']/(platforms.iloc[0]['vis eng']/(platforms.iloc[0]['org imp'] + platforms.iloc[0]['paid imp'])))*100
    elif all_data['platform'] == 'Instagram':
        all_data['platform index'] = (all_data['vis eng rate']/(platforms.iloc[1]['vis eng']/(platforms.iloc[1]['org imp'] + platforms.iloc[1]['paid imp'])))*100
    else:
        all_data['platform index'] = (all_data['vis eng rate']/(platforms.iloc[2]['vis eng']/(platforms.iloc[2]['org imp'] + platforms.iloc[2]['paid imp'])))*100
        
    return all_data['platform index']
    
all_data['platform index'] = all_data.apply(platform_index, axis=1)

In [99]:
themes = all_data.groupby('theme')[['vis eng','org imp','paid imp']].sum().reset_index()
themes

,theme,vis eng,org imp,paid imp


In [ ]:
def theme_index(all_data):
    if all_data['theme'] == '':
        all_data['theme index'] = (all_data['vis eng rate']/(platforms.iloc[0]['vis eng']/(platforms.iloc[0]['org imp'] + platforms.iloc[0]['paid imp'])))*100
    elif all_data['theme'] == '':
        all_data['theme index'] = (all_data['vis eng rate']/(platforms.iloc[1]['vis eng']/(platforms.iloc[1]['org imp'] + platforms.iloc[1]['paid imp'])))*100
    else:
        all_data['theme index'] = (all_data['vis eng rate']/(platforms.iloc[2]['vis eng']/(platforms.iloc[2]['org imp'] + platforms.iloc[2]['paid imp'])))*100
        
    return all_data['theme index']
    
all_data['theme index'] = all_data.apply(theme_index, axis=1)

In [ ]:
all_data = all_data[['platform','date','link','message','media type','content bucket','theme','subtheme','subtheme2','objective','targ aud','CTA','destination','paid','reach','org imp','paid imp','engagement','ER','link clicks','retweets/shares','likes','replies/comments','vis eng','overall index','platform index','theme index','vis eng rate','org video views','paid video views','org video completes','paid video completes','VR','CR']]

In [ ]:
all_data.to_csv('all_data.csv')